# Test sqlite database backend

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!rm lamin.db

In [ ]:
from sqlmodel import Field, Session, SQLModel, create_engine, select
from lamindb.database._sqlite import Dataset, Publication
import json

engine = create_engine("sqlite:///lamin.db")
SQLModel.metadata.create_all(engine)

Write out a JSON schema.

In [ ]:
json.loads(Publication.schema_json(indent=2))

Initialize a database.

In [ ]:
with Session(engine) as session:

    # create publications
    pub_heimberg22 = Publication(name="Heimberg22")
    pub_eraslan21 = Publication(name="Eraslan21")

    # create datasets
    ds_human = Dataset(name="Human lung", publication=pub_heimberg22)
    ds_mouse = Dataset(name="Mouse lung", publication=pub_heimberg22)
    ds_single_nuclei = Dataset(name="Single-nuclei atlas", publication=pub_eraslan21)

    session.add(pub_heimberg22)
    session.add(pub_eraslan21)

    session.commit()

We just learned that we have a new dataset associated to a publication, and want to update it.

In [ ]:
with Session(engine) as session:

    statement = select(Publication).where(Publication.name == "Heimberg22")
    results = session.exec(statement)
    pub_heimberg22 = results.first()

    print("original datasets:")
    print(pub_heimberg22.dataset)

    ds_mouse_last_sample = Dataset(name="Mouse lung last sample")
    pub_heimberg22.dataset.append(ds_mouse_last_sample)

    print("new datasets:")
    print(pub_heimberg22.dataset)